In [2]:
import pandas as pd
import scipy.io
import os
import mne
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [3]:
data = os.path.dirname("./data/")
data_files = os.listdir(data)

In [4]:
# files_5f = [file for file in data_files if "5F" in file]
files_NoMT = [file for file in data_files if "NoMT" in file]
# files_HaLT = [file for file in data_files if "HaLT" in file]
files_FREEFORM = [file for file in data_files if "FREEFORM" in file]
# files_CLA = [file for file in data_files if ("CLA" in file) and (not "SubjectJ" in file)]
# files_CLA_SubjectJ = [file for file in data_files if "CLA-" in file]

In [5]:
mat = scipy.io.loadmat(os.path.join(data, files_NoMT[0]))
mat2=scipy.io.loadmat(os.path.join(data, files_FREEFORM[0]))
print(len(files_NoMT))

mat_NoMT=[scipy.io.loadmat(os.path.join(data, j)) for j in files_NoMT]
mat_FREE=[scipy.io.loadmat(os.path.join(data, j)) for j in files_FREEFORM]

7


In [6]:
ch_names_NoMT = [element[0][0] for element in mat_NoMT[0]["o"][0][0][6]]
ch_names_free = [element[0][0] for element in mat_FREE[0]["o"][0][0][6]]

In [7]:
len(ch_names_free)
#len(ch_names_NoMT)


22

In [8]:
mat_NoMT_df=[pd.DataFrame(i["o"][0][0][5], columns=ch_names_NoMT) for i in mat_NoMT]
mat_FREE_df=[pd.DataFrame(i["o"][0][0][5], columns=ch_names_free) for i in mat_FREE]

In [9]:
for i,v in enumerate(mat_NoMT_df):
    v["State_eGUI"] = mat_NoMT[i]["o"][0][0][4]
for i,v in enumerate(mat_FREE_df):
    v["State_eGUI"] = mat_FREE[i]["o"][0][0][4]

In [10]:
mat_NoMT_df[0]

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,...,F8,T3,T4,T5,T6,Fz,Cz,Pz,X5,State_eGUI
0,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
1,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
2,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
3,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
4,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664395,-15.43,-17.92,-4.51,-6.95,6.61,-7.80,9.05,5.26,10.68,6.48,...,-3.58,7.99,12.88,7.59,-0.31,-14.72,-1.35,22.09,-13.05,0
664396,-33.25,-28.39,-17.48,-16.19,-3.19,0.87,1.61,-0.69,1.72,-1.53,...,-15.19,-5.49,-8.18,-1.70,-9.12,-26.84,-7.27,16.24,-12.42,0
664397,-26.72,-24.98,-16.28,-8.36,-1.30,0.73,0.71,-0.20,2.75,2.29,...,-9.01,-8.90,0.81,-1.39,-4.54,-23.03,-6.11,17.88,-11.86,0
664398,-38.43,-35.16,-24.71,-19.55,-7.39,6.05,-5.85,-2.92,-2.95,-1.85,...,-20.20,-2.05,-13.13,-8.32,-10.93,-30.80,-12.61,12.62,-10.64,0


In [11]:
df_No_MT = pd.DataFrame().append(mat_NoMT_df,ignore_index=True)
df_FREE = pd.DataFrame().append(mat_FREE_df,ignore_index=True)


C:\Users\Patrick\AppData\Local\Temp\ipykernel_19288\4232029754.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_No_MT = pd.DataFrame().append(mat_NoMT_df,ignore_index=True)
C:\Users\Patrick\AppData\Local\Temp\ipykernel_19288\4232029754.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_FREE = pd.DataFrame().append(mat_FREE_df,ignore_index=True)


In [12]:
df_No_MT

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,...,F8,T3,T4,T5,T6,Fz,Cz,Pz,X5,State_eGUI
0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
1,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
2,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
3,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
4,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738795,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
4738796,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
4738797,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0
4738798,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0


In [13]:
df_FREE

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,...,F8,T3,T4,T5,T6,Fz,Cz,Pz,X5,State_eGUI
0,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
1,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
2,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
3,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
4,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152595,-27.64,-54.21,3.60,-0.00,13.48,-13.28,31.17,4.55,33.77,44.05,...,10.83,22.88,24.40,51.07,58.28,3.15,4.44,16.02,-4.07,0
1152596,-42.91,-72.87,0.29,4.35,4.80,-4.99,19.50,-11.35,13.90,28.16,...,-27.49,19.46,21.66,31.61,45.12,-3.71,-3.29,3.84,-4.74,0
1152597,-41.00,-64.72,-3.54,-9.40,6.08,-5.81,24.51,-0.92,18.92,35.24,...,-8.89,30.43,34.07,27.73,51.35,-3.76,-3.28,8.18,-4.42,0
1152598,-28.58,-57.39,7.00,-28.85,9.27,-9.09,22.75,4.41,24.78,33.45,...,10.01,26.01,49.21,26.93,43.35,-1.90,-2.86,9.52,-4.49,0


In [14]:
def epoch_Data(arr,action):
    res=[]
    obj=[]
    for i in tqdm(arr):
        try:
            float(i[-1])
        except ValueError:
            if len(obj)>0:
                 res.append(obj)
            obj=[] 
        else:
       
            if int(i[-1])==action:
                obj.append(i)
            else:
                if len(obj)>0:
                    res.append(obj)
                obj=[] 
    return res
        
        

In [15]:
def make_all_same_length(data,length):
      return [i[0:length] for i in data]
        

In [16]:
def epoch_Data_2(arr,action):
    res=[]
    obj=[]
    for ind,i in tqdm(enumerate(arr)):
        try:
            float(i[-1])
        except ValueError:
            if len(obj)>0:
                 res.append(obj)
            obj=[] 
        else:
                if int(i[-1])==action:
                    if len(obj)==0:
                        obj.extend([*arr[ind-200:ind+1]])
                    else:
                        obj.append(i)
                else:
                    if len(obj)>0:
                        res.append(obj)
                    obj=[] 
    return res
        
        

In [17]:
arr_NoMT=df_No_MT.to_numpy()
arr_FREE=df_FREE.to_numpy()


In [18]:
splitted_nomt=epoch_Data(arr_NoMT,0)
splitted_free=epoch_Data_2(arr_FREE,1)

100%|██████████| 4738800/4738800 [00:03<00:00, 1340075.78it/s]
1152600it [00:00, 1163183.82it/s]


In [19]:
splitted_and_filtered_nomt=[i for i in splitted_nomt if len(i)<500 and len(i)>400]
splitted_and_filtered_free=[i for i in splitted_free if len(i)<500 and len(i)>400]
len(splitted_and_filtered_free)
len(splitted_and_filtered_nomt)

3365

In [20]:
splitted_filtered_cropped_nomt=make_all_same_length(splitted_and_filtered_nomt,400)
splitted_filtered_cropped_free=make_all_same_length(splitted_and_filtered_free,400)


In [21]:
X_nomt=np.array(splitted_filtered_cropped_nomt)
X_nomt=np.swapaxes(X_nomt,1,2)
X_free=np.array(splitted_filtered_cropped_free)
X_free=np.swapaxes(X_free,1,2)

In [22]:
X_nomt.shape

(3365, 23, 400)

In [23]:
X_free.shape

(1066, 23, 400)

In [24]:
X_nomt_train=X_nomt[:3000]
X_nomt_test=X_nomt[3000:]

In [25]:
import random
idy=random.sample(range(0, len(X_free)), 365)
X_free_test=X_free[idy]

In [31]:
from tensorflow.keras import layers
layer = layers.Normalization()
layer1 = layers.Normalization()
layer.adapt(X_nomt_train.astype(float))
layer1.adapt(X_free_test.astype(float))

print(X_nomt_train)
print(np.max(X_nomt_train))
print(np.max(X_free_test))
print(np.max(layer(X_nomt_train)))
print(np.max(layer1(X_free_test)))

[[[ 9.730e+00  6.460e+00  2.620e+00 ...  9.410e+00  1.088e+01  1.376e+01]
  [ 1.515e+01  1.009e+01  5.890e+00 ...  7.910e+00  8.890e+00  4.380e+00]
  [ 9.400e-01 -4.800e-01 -5.800e-01 ...  1.007e+01  1.189e+01  7.300e+00]
  ...
  [ 2.369e+01  2.170e+01  2.175e+01 ...  9.540e+00  1.310e+01  1.717e+01]
  [ 5.400e-01 -2.000e-01  1.100e-01 ... -6.200e-01  6.200e-01  2.700e-01]
  [ 0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00  0.000e+00]]

 [[-8.770e+00 -6.560e+00 -8.670e+00 ... -4.660e+00 -1.740e+00 -4.080e+00]
  [-5.090e+00 -1.060e+00 -7.600e+00 ... -4.770e+00 -3.850e+00 -1.000e+00]
  [-4.740e+00 -4.750e+00 -2.680e+00 ... -7.760e+00 -8.080e+00 -7.050e+00]
  ...
  [-1.760e+00 -2.640e+00 -2.550e+00 ... -1.880e+01 -2.431e+01 -2.687e+01]
  [ 1.800e-01  2.800e-01  4.800e-01 ... -5.000e-02  1.100e-01  2.000e-02]
  [ 0.000e+00  0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00  0.000e+00]]

 [[ 1.061e+01  8.290e+00  1.536e+01 ... -1.769e+01 -1.793e+01 -1.621e+01]
  [ 1.030e+01  1.214e+

In [32]:
latent_dim = 512 
keras.backend.clear_session()
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(1024, activation='gelu'),
      layers.Dense(512, activation='gelu'),
      layers.Dense(64, activation='gelu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(512, activation='gelu'),
      layers.Dense(1024, activation='gelu'),
      layers.Dense(23*400, activation='linear'),
      layers.Reshape((23, 400))
    ])
  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

In [33]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=opt, loss=losses.MeanSquaredError())
autoencoder.fit(layer(X_nomt_train), layer(X_nomt_train),
                epochs=50,
                batch_size=64,
                shuffle=True,
                validation_data=(layer(X_nomt_test[:200]), layer(X_nomt_test[:200])))

Epoch 1/50
47/47 [==============================] - 4s 17ms/step - loss: 0.9126 - val_loss: 0.1013
Epoch 2/50
47/47 [==============================] - 0s 9ms/step - loss: 0.6972 - val_loss: 0.0948
Epoch 3/50
47/47 [==============================] - 0s 10ms/step - loss: 0.5908 - val_loss: 0.0897
Epoch 4/50
47/47 [==============================] - 0s 10ms/step - loss: 0.5145 - val_loss: 0.0859
Epoch 5/50
47/47 [==============================] - 0s 10ms/step - loss: 0.4568 - val_loss: 0.0827
Epoch 6/50
47/47 [==============================] - 0s 10ms/step - loss: 0.4084 - val_loss: 0.0803
Epoch 7/50
47/47 [==============================] - 0s 10ms/step - loss: 0.3746 - val_loss: 0.0794
Epoch 8/50
47/47 [==============================] - 0s 10ms/step - loss: 0.3514 - val_loss: 0.0783
Epoch 9/50
47/47 [==============================] - 0s 9ms/step - loss: 0.3377 - val_loss: 0.0772
Epoch 10/50
47/47 [==============================] - 0s 9ms/step - loss: 0.3184 - val_loss: 0.0767
Epoch 11/50


KeyboardInterrupt: 

In [34]:
err=[]
err2=[]
for i in X_nomt_test:
    encoded = autoencoder.encoder(layer(i)).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(i)-decoded)).mean())
    err.append((np.square(layer(i)-decoded)).mean())
print("###################")
for j in X_free_test:
    encoded = autoencoder.encoder(layer1(j)).numpy()
    decoded = autoencoder.decoder(encoded).numpy()
    #print("Error:",(np.square(layer(j)-decoded)).mean())
    err2.append((np.square(layer1(j)-decoded)).mean())
print("##############")
print(np.array(err).mean())
print(np.array(err2).mean())


###################
##############
0.061190635
0.60914814


descicion 0.65

In [35]:
def acc_Calc(a,b,th):
    first=[1 if i<th else 0 for i in a]
    last=[1 if i>th else 0 for i in b]
    return sum(first+last)/len(first+last)

In [37]:
acc_Calc(err,err2,0.1)

0.9575342465753425

In [102]:
print(err[:10])

[0.050252408, 0.053294912, 0.062432397, 0.04929881, 0.058727484, 0.055836946, 0.06903557, 0.042041827, 0.05590404, 0.05546357]


1,1,1,1,1,1,,0,1,1,1 9/10

In [103]:
print(err2[:10])

[0.09557657, 0.053496804, 0.087779686, 0.059348132, 0.06840686, 0.096350774, 0.06509688, 0.06500423, 0.061500024, 0.05483238]


0,0,1,0,1,1,1,1,0,0 5/10

Acc:

In [104]:
14/20

0.7